In [3]:
from SetCoverPy import setcover
import time
import utm
import stateplane
import pandas as pd
import geopandas as gpd
import shapely as sp
import numpy as np
from geopandas import geoseries
import itertools
from shapely.geometry import Point, MultiPoint

bike = gpd.read_file('/Users/mac/Desktop/citibike/citibike_new2.shp')
demand2 = gpd.read_file('/Users/mac/Desktop/demand/yp_demand.shp')
demand2['geometry'] = demand2['geometry'].str.get(0)
demand = demand2.head(len(demand2))

#demand = demand2.head(100)

OSError: no such file or directory: '/Users/mac/Desktop/citibike/citibike_new2.shp'

In [70]:
#get demand xy-plane
demand_point = []
for index, row in demand.iterrows():
    for pt in list(row['geometry'].coords):
        try:
            x = utm.from_latlon(pt[1], pt[0])[0:2]
            demand_point.append(tuple(x))
        except utm.error.OutOfRangeError:
            pass

#get bike xy-plane
bike_point = []
for index, row in bike.iterrows():
    for pt in list(row['geometry'].coords):
        try:
            x = utm.from_latlon(pt[1], pt[0])[0:2]
            bike_point.append(tuple(x))
        except utm.error.OutOfRangeError:
            pass


new_demand = []   # include all covered demands
bike_demand = []  # include demands to each corresponding bike
new_demand_index = []
for bike in bike_point:
    bike_buffer = Point(bike).buffer(1000)
    for dm in demand_point:
        if bike_buffer.contains(Point(dm)):
            #bike_demand.append(demand)
            if dm not in new_demand:
                new_demand.append(dm)
                new_index = demand_point.index(dm)
                new_demand_index.append(new_index)


covered_demand = [demand.index.isin(new_demand_index)]
covered_demand = demand.loc[demand.index.isin(new_demand_index)]
covered_demand.to_file('/Users/mac/Desktop/covered_demand.shp', driver='ESRI Shapefile')